In [ ]:
import pandas as pd

## PREPROCESSING BEFORE BASELINE

In [ ]:
#Read raw csv used to form baseline model 
flights_data = pd.read_csv('/content/Regression_no_outliers_cleaned_data.csv')

In [ ]:
flights_data.shape

(173391, 24)

In [ ]:
flights_grouped = flights_data.groupby(by='op_unique_carrier').mean()
flights_grouped.reset_index(inplace=True)
flights_grouped['total_delay'] = flights_grouped['carrier_delay'] + flights_grouped['nas_delay'] + flights_grouped['late_aircraft_delay']
flights_grouped = flights_grouped.drop(columns = ['dep_delay','op_carrier_fl_num','crs_dep_time','crs_elapsed_time','arr_delay','dep_time','arr_time','crs_arr_time','actual_elapsed_time','distance','carrier_delay','nas_delay','security_delay','late_aircraft_delay','weather_delay','year','month','day','weekday'])

flights_grouped

,op_unique_carrier,total_delay
0,9E,11.991362
1,AA,12.814187
2,AS,9.857995
3,AX,19.822542
4,B6,19.228117
5,C5,27.212379
6,CP,12.383745
7,DL,8.994286
8,EV,14.181963
9,F9,18.056772


In [ ]:
#Merge averages dataframe with original 
flights_data = flights_data.merge(flights_grouped, on = 'op_unique_carrier', how = 'outer')


In [ ]:
flights_data = flights_data.drop(columns = ['fl_date','mkt_unique_carrier','dest','dep_delay','dep_time','arr_time','nas_delay','security_delay','weather_delay','late_aircraft_delay','carrier_delay'])
flights_data

,op_unique_carrier,op_carrier_fl_num,origin,crs_dep_time,arr_delay,crs_arr_time,crs_elapsed_time,actual_elapsed_time,distance,year,month,day,weekday,total_delay
0,B6,112,ORD,2033,-10.0,2350.0,137.0,144.0,867.0,2018.0,3.0,6.0,0.0,19.228117
1,B6,1837,BOS,2041,-7.0,2247.0,126.0,134.0,632.0,2019.0,3.0,24.0,1.0,19.228117
2,B6,790,DCA,1535,7.0,1711.0,96.0,78.0,399.0,2018.0,9.0,1.0,1.0,19.228117
3,B6,476,MCO,2045,7.0,2326.0,161.0,164.0,1072.0,2018.0,9.0,24.0,0.0,19.228117
4,B6,1979,DCA,1655,-9.0,1939.0,164.0,154.0,899.0,2019.0,3.0,31.0,1.0,19.228117
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173386,VX,1938,SFO,1859,26.0,2039.0,100.0,100.0,337.0,2018.0,1.0,22.0,0.0,9.487179
173387,VX,1168,LAX,2210,-6.0,629.0,319.0,277.0,2454.0,2018.0,2.0,16.0,0.0,9.487179
173388,VX,1367,BOS,1635,-34.0,2020.0,405.0,371.0,2611.0,2018.0,3.0,5.0,0.0,9.487179
173389,VX,1388,SEA,1345,19.0,1600.0,135.0,125.0,679.0,2018.0,1.0,10.0,0.0,9.487179


In [ ]:
# from sklearn.preprocessing import LabelEncoder
# le = LabelEncoder()
# flights_data["origin"]= le.fit_transform(flights_data["origin"])

#Did not end up using this

In [ ]:
flights_data.dtypes

op_unique_carrier       object
op_carrier_fl_num        int64
origin                  object
crs_dep_time             int64
arr_delay              float64
crs_arr_time           float64
crs_elapsed_time       float64
actual_elapsed_time    float64
distance               float64
year                   float64
month                  float64
day                    float64
weekday                float64
total_delay            float64
dtype: object

In [ ]:
# missing data
total = flights_data.isnull().sum().sort_values(ascending=False)
percent = (flights_data.isnull().sum()/flights_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
weekday,1,0.000006
day,1,0.000006
month,1,0.000006
year,1,0.000006
distance,1,0.000006
actual_elapsed_time,1,0.000006
crs_elapsed_time,1,0.000006
crs_arr_time,1,0.000006
arr_delay,1,0.000006
total_delay,0,0.000000


In [ ]:
flights_data = flights_data.dropna()

In [ ]:
# missing data
total = flights_data.isnull().sum().sort_values(ascending=False)
percent = (flights_data.isnull().sum()/flights_data.isnull().count()).sort_values(ascending=False)
missing_data = pd.concat([total, percent], axis=1, keys=['Total', 'Percent'])
missing_data.head(20)

,Total,Percent
total_delay,0,0.0
weekday,0,0.0
day,0,0.0
month,0,0.0
year,0,0.0
distance,0,0.0
actual_elapsed_time,0,0.0
crs_elapsed_time,0,0.0
crs_arr_time,0,0.0
arr_delay,0,0.0


## Baseline Model #1 Random Forest

In [ ]:
X = flights_data[['crs_arr_time','crs_elapsed_time','distance','year','month','day','total_delay']]
y = flights_data['arr_delay']

In [ ]:
from sklearn.model_selection import train_test_split
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [ ]:
#Import Random Forest Model
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
import numpy as np

In [ ]:
#Fitting against itself
reg = RandomForestRegressor(n_estimators=100)

reg.fit(X_train,y_train)

y_pred=reg.predict(X_test)



In [ ]:
 rmse = np.sqrt(mean_squared_error(y_test, y_pred))
 rmse

43.724118872502515

In [ ]:
reg.score(X_test,y_test)

-0.030470154574230968

## Dummy Regressor 

In [ ]:
from sklearn.dummy import DummyRegressor

dumreg = DummyRegressor(strategy='mean')
dumreg2 = DummyRegressor(strategy='median')
dumreg.fit(X_train,y_train)
dumreg2.fit(X_train,y_train)


DummyRegressor(strategy='median')

In [ ]:
y_pred = dumreg.predict(X_test)
y_pred2 = dumreg.predict(X_test)

In [ ]:
dumreg.score(y_test,y_pred)
#dumreg2.score(y_test,y_pred2)

1.0

## Linear Regression

In [ ]:
from sklearn.linear_model import LinearRegression

In [ ]:
from sklearn.metrics import mean_absolute_error

In [ ]:
from sklearn.metrics import r2_score

In [ ]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X=scaler.fit_transform(X)


In [ ]:
from sklearn.model_selection import train_test_split
# Split dataset into training set and test set
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3) # 70% training and 30% test

In [ ]:
# creating linear regression
lr = LinearRegression()
lr.fit(X_train,y_train)
y_lr = lr.predict(X_test)

In [ ]:
MAElr = mean_absolute_error(y_test,y_lr)
MAElr

24.182006526422033

In [ ]:
R2lr = r2_score(y_test,y_lr)
R2lr

0.014529147304010892